In [1]:
import os
import sys

import warnings
warnings.filterwarnings("ignore")

os.environ['environment'] = 'interactive'
os.environ['UTILS_PATH'] = '/Users/arsenijkadaner/Yandex.Disk.localized/master/code/git/python-utils-ak'
os.environ['PROJECT_PATH'] = '/Users/arsenijkadaner/Yandex.Disk.localized/master/code/git/2020.10-umalat/umalat' 
os.environ['SQLITE_PATH'] = '/Users/arsenijkadaner/Yandex.Disk.localized/master/code/git/2020.10-umalat/umalat/data.sqlite' 
sys.path.append(os.environ['UTILS_PATH'])
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

WORKING IN INTERACTIVE MODE
WORKING IN INTERACTIVE MODE


In [2]:
from utils_ak.block_tree import *

In [3]:
def make_boiling(boiling_model):
    maker, make = init_block_maker("boiling", boiling_id=1, boiling_label='рикотта') # todo: set boiling_id from inputs
    
    bt = boiling_model.boiling_technology
    make('heating', size=(bt.heating_time // 5, 0))
    make('delay', size=(bt.delay_time // 5, 0))
    make('protein_harvest', size=(bt.protein_harvest_time // 5, 0))
    make('abandon', size=(bt.abandon_time // 5, 0))
    make('pumping_out', size=(bt.pumping_out_time // 5, 0))

    return maker.root

In [4]:
boiling_model = cast_model(RicottaBoiling, 9)
make_boiling(boiling_model)

======== boiling (0, 8] x (0, 0]
=== heating (0, 3] x (0, 0]
   = delay (3, 4] x (0, 0]
    == protein_harvest (4, 6] x (0, 0]
      = abandon (6, 7] x (0, 0]
       = pumping_out (7, 8] x (0, 0]

In [5]:
def make_boiling_sequence(sku):
    boiling_model = sku.made_from_boilings[0]
    maker, make = init_block_maker("boiling_sequence")
    n_boilings = 3 # todo: take from sku
    boilings = [make_boiling(boiling_model) for _ in range(n_boilings)]  
    
    for b_prev, b in SimpleIterator(boilings).iter_sequences(2, method="any"):
        if not b:
            continue

        if not b_prev:
            push(maker.root, b, push_func=add_push)
        else:
            b.props.update(x=(b_prev['delay'].x[0], 0))
            push(maker.root, b, push_func=add_push)
    return maker.root


In [6]:
sku = cast_model(RicottaSKU, 62)
make_boiling_sequence(sku)

============== boiling_sequence (0, 14] x (0, 0]
======== boiling (0, 8] x (0, 0]
=== heating (0, 3] x (0, 0]
   = delay (3, 4] x (0, 0]
    == protein_harvest (4, 6] x (0, 0]
      = abandon (6, 7] x (0, 0]
       = pumping_out (7, 8] x (0, 0]
   ======== boiling (3, 11] x (0, 0]
   === heating (3, 6] x (0, 0]
      = delay (6, 7] x (0, 0]
       == protein_harvest (7, 9] x (0, 0]
         = abandon (9, 10] x (0, 0]
          = pumping_out (10, 11] x (0, 0]
      ======== boiling (6, 14] x (0, 0]
      === heating (6, 9] x (0, 0]
         = delay (9, 10] x (0, 0]
          == protein_harvest (10, 12] x (0, 0]
            = abandon (12, 13] x (0, 0]
             = pumping_out (13, 14] x (0, 0]

In [7]:
def make_boiling_group(sku):
    kg = 50 # todo: rake from input
    maker, make = init_block_maker("boiling_group")
    boiling_sequence = make_boiling_sequence(sku)
    push(maker.root, boiling_sequence)
    analysis_start = listify(boiling_sequence['boiling'])[-1]['abandon'].x[0]
    with make('analysis', x=(analysis_start, 0), push_func=add_push):
        analysis = cast_model(RicottaAnalysisTechnology, 1) # todo: take from boiling_model
        make('preparation', size=(analysis.preparation_time // 5, 0))
        make('analysis', size=(analysis.preparation_time // 5, 0))
        make('pumping', size=(analysis.preparation_time // 5, 0))

    packing_start = maker.root['analysis']['pumping'].x[0] + 1
    packing_time = custom_round(kg / sku.packing_speed * 60, 5, 'ceil')

    make('packing', x=(packing_start, 0), size=(packing_time // 5, 0), push_func=add_push)
    return maker.root

In [8]:
make_boiling_group(sku)

============================= boiling_group (0, 29.0] x (0, 0]
============== boiling_sequence (0, 14] x (0, 0]
======== boiling (0, 8] x (0, 0]
=== heating (0, 3] x (0, 0]
   = delay (3, 4] x (0, 0]
    == protein_harvest (4, 6] x (0, 0]
      = abandon (6, 7] x (0, 0]
       = pumping_out (7, 8] x (0, 0]
   ======== boiling (3, 11] x (0, 0]
   === heating (3, 6] x (0, 0]
      = delay (6, 7] x (0, 0]
       == protein_harvest (7, 9] x (0, 0]
         = abandon (9, 10] x (0, 0]
          = pumping_out (10, 11] x (0, 0]
      ======== boiling (6, 14] x (0, 0]
      === heating (6, 9] x (0, 0]
         = delay (9, 10] x (0, 0]
          == protein_harvest (10, 12] x (0, 0]
            = abandon (12, 13] x (0, 0]
             = pumping_out (13, 14] x (0, 0]
            ====== analysis (12, 18] x (0, 0]
            == preparation (12, 14] x (0, 0]
              == analysis (14, 16] x (0, 0]
                == pumping (16, 18] x (0, 0]
                 ============ packing (17, 29.0] x (0,

In [9]:
validator = ClassValidator(window=2)


def validate(b1, b2):
    for line_num in range(3):
        if (
            line_num not in b1.props["line_nums"]
            or line_num not in b2.props["line_nums"]
        ):
            continue
        
        boiling1 = listify(b1['boiling_sequence']['boiling'])[b1.props['line_nums'].index(line_num)]
        boiling2 = listify(b2['boiling_sequence']['boiling'])[b2.props['line_nums'].index(line_num)]

        validate_disjoint_by_axis(boiling1, boiling2)
    
    validate_disjoint_by_axis(b1['analysis'], b2['analysis'])
    validate_disjoint_by_axis(b1['packing'], b2['packing'])
    
validator.add("boiling_group", "boiling_group", validate)

In [10]:
def make_schedule():
    maker, make = init_block_maker("schedule")

    boiling_groups = [make_boiling_group(sku) for _ in range(2)]
    for bg in boiling_groups:
        push(
            maker.root,
            bg,
            push_func=AxisPusher(start_from="last_beg"),
            validator=validator,
            iter_props=[{"line_nums": v} for v in [[0, 1, 2], [1, 2, 0], [2, 0, 1]]],
        )
    return maker.root

In [11]:
make_schedule()

2021-03-09 15:20:22.161 | DEBUG    | utils_ak.block_tree.pushers.iterative:update:106 - Disposition
2021-03-09 15:20:22.164 | DEBUG    | utils_ak.block_tree.pushers.iterative:update:106 - Disposition
2021-03-09 15:20:22.168 | DEBUG    | utils_ak.block_tree.pushers.iterative:update:106 - Disposition
2021-03-09 15:20:22.173 | DEBUG    | utils_ak.block_tree.pushers.iterative:update:106 - Disposition
2021-03-09 15:20:22.178 | DEBUG    | utils_ak.block_tree.pushers.iterative:update:106 - Disposition


========================================= schedule (0, 41.0] x (0, 0]
============================= boiling_group (0, 29.0] x (0, 0]
============== boiling_sequence (0, 14] x (0, 0]
======== boiling (0, 8] x (0, 0]
=== heating (0, 3] x (0, 0]
   = delay (3, 4] x (0, 0]
    == protein_harvest (4, 6] x (0, 0]
      = abandon (6, 7] x (0, 0]
       = pumping_out (7, 8] x (0, 0]
   ======== boiling (3, 11] x (0, 0]
   === heating (3, 6] x (0, 0]
      = delay (6, 7] x (0, 0]
       == protein_harvest (7, 9] x (0, 0]
         = abandon (9, 10] x (0, 0]
          = pumping_out (10, 11] x (0, 0]
      ======== boiling (6, 14] x (0, 0]
      === heating (6, 9] x (0, 0]
         = delay (9, 10] x (0, 0]
          == protein_harvest (10, 12] x (0, 0]
            = abandon (12, 13] x (0, 0]
             = pumping_out (13, 14] x (0, 0]
            ====== analysis (12, 18] x (0, 0]
            == preparation (12, 14] x (0, 0]
              == analysis (14, 16] x (0, 0]
                == pumping (1

In [12]:
def _make_frontend_boiling(boiling):
    maker, make = init_block_maker(
        "boiling",
        axis=1,
        size=(0, 3),
        boiling_id=boiling.props["boiling_id"],
        boiling_label=boiling.props["boiling_label"],
    )

    with make():
        make("boiling_num", size=(boiling["heating"].size[0], 1))
        make("boiling_name", size=(boiling.size[0] - boiling["heating"].size[0], 1))
    return maker.root

In [13]:
_make_frontend_boiling(make_boiling(boiling_model))

======== boiling (0, 8] x (0, 3]
======== block (0, 8] x (0, 1]
=== boiling_num (0, 3] x (0, 1]
   ===== boiling_name (3, 8] x (0, 1]

In [14]:
def make_frontend(schedule):
    maker, make = init_block_maker("frontend", axis=1)
    make("stub", size=(0, 1)) # start with 1

    boiling_lines = []
    for i in range(3):
        boiling_lines.append(
            make(f"boiling_line_{i}", size=(0, 3), is_parent_node=True).block
        )
        make("stub", size=(0, 2))

    for boiling_group in listify(schedule['boiling_group']):
        for i, line_num in enumerate(boiling_group.props['line_nums']):
            boiling = listify(boiling_group['boiling_sequence']['boiling'])[i]
            push(boiling_lines[line_num], _make_frontend_boiling(boiling))
    return maker.root

In [15]:
schedule = make_schedule()
frontend = make_frontend(schedule)
frontend

2021-03-09 15:20:22.255 | DEBUG    | utils_ak.block_tree.pushers.iterative:update:106 - Disposition
2021-03-09 15:20:22.258 | DEBUG    | utils_ak.block_tree.pushers.iterative:update:106 - Disposition
2021-03-09 15:20:22.261 | DEBUG    | utils_ak.block_tree.pushers.iterative:update:106 - Disposition
2021-03-09 15:20:22.266 | DEBUG    | utils_ak.block_tree.pushers.iterative:update:106 - Disposition
2021-03-09 15:20:22.271 | DEBUG    | utils_ak.block_tree.pushers.iterative:update:106 - Disposition


================ frontend (0, 16] x (0, 16]
================ boiling_line_0 (0, 16] x (1, 4]
======== boiling (0, 8] x (1, 4]
======== block (0, 8] x (1, 2]
=== boiling_num (0, 3] x (1, 2]
   ===== boiling_name (3, 8] x (1, 2]
        ======== boiling (8, 16] x (1, 4]
        ======== block (8, 16] x (1, 2]
        === boiling_num (8, 11] x (1, 2]
           ===== boiling_name (11, 16] x (1, 2]
================ boiling_line_1 (0, 16] x (6, 9]
======== boiling (0, 8] x (6, 9]
======== block (0, 8] x (6, 7]
=== boiling_num (0, 3] x (6, 7]
   ===== boiling_name (3, 8] x (6, 7]
        ======== boiling (8, 16] x (6, 9]
        ======== block (8, 16] x (6, 7]
        === boiling_num (8, 11] x (6, 7]
           ===== boiling_name (11, 16] x (6, 7]
================ boiling_line_2 (0, 16] x (11, 14]
======== boiling (0, 8] x (11, 14]
======== block (0, 8] x (11, 12]
=== boiling_num (0, 3] x (11, 12]
   ===== boiling_name (3, 8] x (11, 12]
        ======== boiling (8, 16] x (11, 14]
        ===

In [16]:
RICOTTA_STYLE = {
    'boiling_num': {"text": "{boiling_id}"},
    'boiling_name': {"text": "{boiling_label}"},
        "stub": {"visible": False},
}

In [17]:
draw_excel_frontend

<function app.schedule_maker.frontend.draw_excel_frontend(frontend, style, open_file=False, fn='schedule.xlsx')>

In [18]:
draw_excel_frontend(frontend, RICOTTA_STYLE, open_file=True)